# A/B Testing  

This notebook contains my notes on A/B Testing from Udacity's free online course as well as books like Lean Analytics, Lean Product Playbook and Trustworthy Online Experiments: A Practical Guide to A/B Testing (TOE). 

## Definition:

In simple terms, **A/B tests** or **Online Controlled Experiments (OCEs)** are mechanisms to test new product features, backend changes, UI changes or any strategic decisions by implementing the change only on a subset of users, comparing it to the status quo or other alternatives and analyzing the effect on chosen metrics to determine the impact before officially rolling it out. 

A/B Tests are best used when introducing enhancements or improvements to products with an existing user base (an available population to conduct tests on). Introduction of new products could benefit more from qualitative methods like consumer interviews and surveys. 

## Some considerations before using A/B Testing on your product:

Before deciding whether to use A/B testing in your own business, it would be a good idea to ask these questions:

- Do you have an **experimental unit** to test on? - like users, cookies, etc. 
- Do you have enough units to test on? - TOE recommends thousands of experimental units for the experiment to be useful.
- Do you have a key metric(s), aka **Overall Evaluation Criteria**? - a goal that is agreed upon and can be practically evaluated. 
- Are the changes you want to test easy to make? - you would need to weigh the costs of making software changes against the benefits potentially realised from the test.
- For UX testing - do you have repeat users whose experience (in terms of satisfaction, engagement, etc) can be significantly evaluated or improved? - testing impact of changes on engagement for one-off purchase websites like home rentals may not be as effective as say a social network or e-commerce site. 

## Components of an A/B Test:

TOE defines the components of an A/B test as follows:

- **_Parameter_**: Aka experimental variable/factor. This is what determines the difference between the A and B groups of your test. In case of a simple UI change to the colour of the checkout button, the parameter can be defined as colour. The original colour is often assigned to group A - known as the **Control Group** and the new colour(s) are assigned the the other groups - known as the **Treatment(s)**. Often times, multiple variations of the parameter are tested in a single test in a process known as **Multivariate testing** 


- **_Variant_**: This is simply another name for the group assignment - Treatment vs Control or A/B/C... It is a function of the parameter or experimental variable. The distinction between parameter and variant is more apparent when there are 2 or more experimental variables interacting. For example if we are testing for both colour and shape (m and n of each) of a button, we have 2 parameters - colour and shape. We have m times n variants - each combination of colour and shape. 


- **_Randomization Unit_**: This is also often called the _unit of diversion_. It is the unit on which a group (Treatment or Control) is randomly assigned. The most common unit (especially for user visible changes) is user id. However, for some non user visible (like backend changes) cookie or page view based diversion may also be applied. 



## Metrics

This warrants a whole section to itself because choosing the right metric can change the whole inference and impact of your experiment. 

- Choosing the right metrics 
- Normalizing metrics - size may vary across variants 
- Some common metrics for different business types 
- Variability


## Statistics for Online Controlled Experiments 

In the this section notebook, I will go through some of the common statistical methods involved in planning and analysing A/B Tests as well as some snippets of code used to obtain results.

#### Sizing and Power Analysis 

In order for us to make robust decisions based on A/B Tests, we must make sure that the numbers we observe in the test are reliable. The **power** of a test is defined as the probability that we will observe a significant difference in the metric between test groups given there actually exists such a difference. Having a high enough power means that we are not missing out on significant differences (even if they are small). How "powerful" a test is is dependent on the sample size.

Another important concept while considering power and sizing is the **minimum detectable effect (MDE)**, aka **practical significance level**. This is the change in the metric we would like to see for it to actually have some impact on our business. If we think that a 2% absolute change in click-through-rate as a result of a change in shape of a button makes it worth implementing, then this is our MDE. It can be specified both in absolute or in relative terms. In order for this to make sense, we must also specify our **baseline** probability or the expected value of the metric in the Control group. 

Finally the concepts of **statistical significance** or **confidence level** from hypothesis testing applies directly to A/B tests as well. 

Having determined these factors, the sample size can be calculated using a formula specified in various statistics text books. 



Here is the R code to calculate the sample size given some parameters. 


In [4]:
#Sample Size Calculations 

#inputs 
power <- 80
conflevel <- 95
baseline <- 8
effectsize <- 2
relative <- F

#function to calculate sample size 
sample_size <- function(power, conflevel, baseline, effectsize, relative){
    if (relative == T){
        effectsize <- (effectsize/100)*(baseline)
    }
    else {
        effectsize <- effectsize
    }
    p1 <- baseline/100
    p2 <- p1 - effectsize/100
    p <- (p1+p2)/2
    q <- 1 - p
    q1 <- 1 - p1
    q2 <- 1 - p2
    alpha <- 1 - conflevel/100
    n <- (sqrt(p*q*(2))*qnorm(1-alpha/2) + sqrt(p1*q1 + p2*q2)*qnorm(power/100))^2/(effectsize/100)^2
    return(round(n))
}

In [5]:
sample_size(power, conflevel, baseline, effectsize, relative)

[1] 2554

More information about the formula used here can be found in this [article](https://towardsdatascience.com/required-sample-size-for-a-b-testing-6f6608dd330a) published by Towards Data Science. There are also various online sample size calculators that provide close estimates. 

- [Optimizely](https://www.optimizely.com/sample-size-calculator/)
- [EvanMiller.org](https://www.evanmiller.org/ab-testing/sample-size.html)

I have also created my own sample size calculator on RShiny which can be accessed by running runGitHub( "A-B-Testing-Toolkit", "preranahiriyur") on RStudio with RShiny package installed. 


This VWO [article](https://vwo.com/blog/how-to-calculate-ab-test-sample-size/) explains a few ways to calculate sample sizes. 
                                                                        

#### Common distributions - Binomial and Normal Distribution - (common metrics and their distributions, a table)

Depending on the randomization unit and what we're measuring, we may observe different statistical distributions in our data. For example, while measuring click-through-probability, we might assume that the variable follows a _Binomial Distribution_ with x = 1 if clicked and x = 0 if not. 

The table below mentions the statistical distributions

| Metric | Distribution | 
| --- | --- | 
| Probability | Binomial (Approx normal for large sample) | 
| Mean | Normal (Central Limit Theorem) |
| Rates | Poisson |
| Counts | Maybe Normal |

In case you are interested in the properties of some of these distributions, you may find this [article](https://www.analyticsvidhya.com/blog/2017/09/6-probability-distributions-data-science/#:~:text=Any%20distribution%20is%20known%20as,under%20the%20curve%20is%201) useful.


#### Hypothesis testing 

Knowing the properties of the underlying distribution of the data can help inform analysis. If the data is approximately normally distributed or has asymptotically normal properties, then we can use a simple hypothesis test. 

A hypothesis test has the following components:

- **Null hypothesis** ($H_0$): An assumption or a statement you would like to test against. It is often stated as an equality, say that the click-through-rate for groups A and B are equal. 

- **Alternative hypothesis**: This is what we would like to prove, often stated as the opposite of the null. In this case it could be that the click-through-rate in group B is simply not equal to A. We could also say the former is greater than or lesser than the latter. All of these are valid alternative hypotheses. 

- **Test statistic**: This is a function of the observed value of the metric on our data. It is used together with the critical value to make a decision about the null and alternative hypotheses. 

- **Critical value**: This is a cut-off value based on our Null hypothesis and assumptions about the underlying distribution of the data. We tend to **reject** the null when our test statistic exceeds this value. 

- **p-value**: This is the probability of observing something similar to or more extreme than the metric we have observed given our assumption that the null hypothesis is true. A low p-value means that it is unlikely we may have oberved this value for the metric if the null were true, suggesting we reject the null. A high p-value means that it is highly probable that we observe such a value for the metric given the null and there we are unable to reject it. 


As an example of a simple hypothesis test, I will use data from Kevin Hillstrom's MineThatData E-Mail Analytics And Data Mining Challenge. More info on the data can be found [here](https://blog.minethatdata.com/2008/03/minethatdata-e-mail-analytics-and-data.html?m=1)



In [22]:
email_data <- read.csv("Kevin_Hillstrom_MineThatData_E_MailAnalytics_DataMiningChallenge.csv", header=T)
head(email_data, 5)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
,<int>,<chr>,<dbl>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<dbl>
1,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0
2,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0
3,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0
4,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0
5,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0


In [23]:
#Let's do an A/B Test to see if the visit rate among Men's E-Mail Campaign receivers and No-Email Campaign receivers are equal

no_email <- nrow(email_data[email_data$segment=="No E-Mail", ])
mens_email <- nrow(email_data[email_data$segment=="Mens E-Mail", ])
no_email_visit <- nrow(email_data[email_data$segment=="No E-Mail" & email_data$visit == 1, ])
mens_email_visit <- nrow(email_data[email_data$segment=="Mens E-Mail" & email_data$visit == 1, ])

x <- c(no_email_visit, mens_email_visit)
n <- c(no_email, mens_email)

prop.test(x,n, correct=F)


	2-sample test for equality of proportions without continuity
	correction

data:  x out of n
X-squared = 505.62, df = 1, p-value < 2.2e-16
alternative hypothesis: two.sided
95 percent confidence interval:
 -0.08322563 -0.06995350
sample estimates:
   prop 1    prop 2 
0.1061673 0.1827568 


The results above tell us that the p-value for this test is less than 2.2e-16 which is very small. The default null is that the visit rates are equal which we can reject based on this p-value. 

##### Non-parametric methods

Sometimes our metrics may be defined as ratios or medians, that may not follow a common statistical distribution. Using the properties of some distribution in these cases may lead to wrong analysis. In such scenarios where we cannot be sure of the underlying distribution of our data, it is recommended to use **Non-parametric** methods such as the Wilcoxon signed rank test or the Mann-Whitney test to test hypotheses. 

In [25]:
#Try running
#help("wilcox.test")

#### Variability 



- Empirical vs Analytical estimates of variability and confidence intervals 
- A/A Tests 
- Sensitivity vs Robustness 
- Analyzing Ratios 

## Designing an Experiment 

- Unit of diversion - considerations 
- Size - considerations 
- Population vs Cohort vs Target Population
- Duration vs Exposure - considerations to limit exposure 

## Ethics of A/B Testing and Online Experimentation

## Common Pitfalls 